In [1]:
import numpy as np
from tqdm import tqdm_notebook
import itertools

N = 20 ## Cambiar lectura de matriz
k = N-1
covmat = np.random.rand(N,N) #Aca corresponde ingresar la matriz de covarianzas ya calculada
covmat[[range(0,N)],[range(0,N)]] = 1 #Esta linea esta demas, eliminar cuando se vea como recibir la matriz desde matlab
arr = list(range(0,N)) #Arreglo para generar las mascaras de las submatrices cuadradas

In [2]:
covmat

array([[1.00000000e+00, 6.76664913e-01, 9.04095981e-01, 6.48070656e-01,
        2.56875742e-01, 9.66207519e-01, 1.80357954e-02, 3.94327195e-01,
        8.96267471e-01, 8.83892246e-01, 1.98004183e-01, 5.81732495e-01,
        5.27905957e-01, 5.74025502e-01, 8.53021925e-02, 3.73014425e-01,
        8.27988828e-01, 7.46751376e-01, 5.18822227e-01, 1.01983740e-01],
       [6.99108471e-01, 1.00000000e+00, 9.82291393e-01, 6.15210894e-01,
        9.83748885e-01, 2.41365043e-01, 7.06051838e-01, 4.83938121e-01,
        5.69519423e-01, 3.84632202e-01, 6.43775063e-01, 5.52688842e-01,
        2.89394031e-01, 7.41643930e-01, 2.53529477e-01, 3.13701375e-01,
        5.56966913e-01, 5.31633009e-01, 9.32614665e-01, 4.28674254e-01],
       [7.09669853e-01, 1.38778439e-01, 1.00000000e+00, 7.35250932e-01,
        6.55942335e-01, 8.15533486e-02, 9.96824935e-01, 5.13452220e-01,
        8.78144457e-01, 8.09211155e-01, 3.92173093e-03, 8.52098949e-01,
        7.22508031e-04, 6.31479911e-01, 1.03797504e-01, 9.2038

In [3]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster') #Cambiar por cluster brincolab
rc.purge_everything()  #Limpiar la sesion

TimeoutError: Hub connection request timed out

In [ ]:
dview = rc[:] # use all engines

In [ ]:
dview['covmat'] = covmat #Distribuimos la matriz de covarianza

In [6]:
dview.block = True #Modo block

In [7]:
from math import factorial, ceil
from itertools import islice, chain

def batch(iterable, size):
    sourceiter = iter(iterable)
    while True:
        batchiter = islice(sourceiter, size)
        yield chain([next(batchiter)], batchiter)

iteraciones = {}
det_calcular = {}
indices_scatter = {}
for i in range(2,N):
    iteraciones[i] = itertools.combinations(arr, i)
    iters = itertools.combinations(arr, i)
    n = int(factorial(N) / factorial(i) / factorial(N-i))
    n_per_process = ceil(int(n)/len(rc.ids) )
    print(i, n, n_per_process)
    det_calcular[i] = (n,n_per_process)
    indexes = []
    for j in range(0,n,n_per_process):
        sl = islice(iters, j , j + n_per_process )
        indexes.append(sl)
    """batchiter = batch(iteraciones[i], int(ceil(n/len(rc.ids))))
    for j in range(len(rc.ids)):
        indexes.append(next(batchiter))"""
        
    indices_scatter[i] = indexes

2 190 4
3 1140 20
4 4845 84
5 15504 268
6 38760 669
7 77520 1337
8 125970 2172
9 167960 2896
10 184756 3186
11 167960 2896
12 125970 2172
13 77520 1337
14 38760 669
15 15504 268
16 4845 84
17 1140 20
18 190 4
19 20 1


In [8]:
n = int(factorial(N) / factorial(2) / factorial(N-2))
n_per_process = ceil(int(n)/len(rc.ids))
print(n,n_per_process)
iters = itertools.combinations(arr, 2)
indexes = []
for i in range(0,190,4):
    sl = islice(iters,0,4)
    indexes.append(sl)

190 4


In [9]:
len(indexes)

48

In [10]:
"""i = 0
j = 0
for el in indices_scatter[2]:
    print(el)
    for subel in el:
        print(subel)
        j+=1
    print("=====")
    i+=1
i,j"""

'i = 0\nj = 0\nfor el in indices_scatter[2]:\n    print(el)\n    for subel in el:\n        print(subel)\n        j+=1\n    print("=====")\n    i+=1\ni,j'

In [11]:
indices_scatter[2]

In [12]:
idx = 2
iteraciones[idx] = itertools.combinations(arr, idx)
n = int(factorial(N) / factorial(idx) / factorial(N-idx))

In [13]:
batchiter = batch(iteraciones[idx], int(ceil(n/len(rc.ids))))

In [14]:
it = next(batchiter)
for el in it:
    print(el)

(0, 1)
(0, 2)
(0, 3)
(0, 4)


In [15]:
i = 0
for el in batchiter:
    #print(el)
    i+=1
    for sub_el in el:
        pass
        #print(sub_el)
i

/home/srodriguez/miniconda/envs/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: generator 'batch' raised StopIteration
  


47

In [16]:
iteraciones

{2: <itertools.combinations at 0x7f8f50d35b88>,
 3: <itertools.combinations at 0x7f8f50d23228>,
 4: <itertools.combinations at 0x7f8f50d114f8>,
 5: <itertools.combinations at 0x7f8f50d15818>,
 6: <itertools.combinations at 0x7f8f50d19b38>,
 7: <itertools.combinations at 0x7f8f50d17e58>,
 8: <itertools.combinations at 0x7f8f50d1c1d8>,
 9: <itertools.combinations at 0x7f8f50d22548>,
 10: <itertools.combinations at 0x7f8f50d1f8b8>,
 11: <itertools.combinations at 0x7f8f50d25c28>,
 12: <itertools.combinations at 0x7f8f50d26f48>,
 13: <itertools.combinations at 0x7f8f50d282c8>,
 14: <itertools.combinations at 0x7f8f50d295e8>,
 15: <itertools.combinations at 0x7f8f50d2a908>,
 16: <itertools.combinations at 0x7f8f50d2bc28>,
 17: <itertools.combinations at 0x7f8f50d2ef48>,
 18: <itertools.combinations at 0x7f8f50d31278>,
 19: <itertools.combinations at 0x7f8f50d32278>}

In [17]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


In [18]:
def det(N):
    dets = numpy.zeros(N,dtype=numpy.float32)
    if len(index_mat) > 0:
        idx_mat = index_mat[0]
    else:    
        return {'dets':[],'matrices':[],'idx':[]}
    matrices = []
    idxs = []
    i=0
    for ind in idx_mat:
        idx = list(ind)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        matrices.append(matrix)
        idxs.append(idx)
        i += 1
    return {'dets':dets,'idx':idxs}

In [19]:
responses = {}
for i in range(2,N):
    n_dets = det_calcular[i][0]
    n_scatters = det_calcular[i][1]
    print(n_scatters)
    if n_scatters > 0:
        dview.scatter('index_mat',indices_scatter[i])
        ar = dview.apply_async(det,n_scatters)
        ar.wait()
        responses[i] = ar.get()
#for el in hola2:
#    print(el)

4
20
84
268
669
1337
2172
2896
3186
2896
2172
1337
669
268
84
20
4
1


In [26]:
responses[18][20]

{'dets': array([ 0.01108893, -0.01133634, -0.01925328,  0.0073041 ], dtype=float32),
 'idx': [[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19],
  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19],
  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19],
  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19]]}

In [110]:
covmat[[0, 3, 8]]

array([[1.        , 0.25654559, 0.13967424, 0.69208183, 0.70448127,
        0.72790431, 0.11672791, 0.1618712 , 0.00132104, 0.6310362 ,
        0.61708706, 0.79560084, 0.7265054 , 0.3064654 , 0.10317961,
        0.18111312, 0.22340437, 0.96763108, 0.38655911, 0.71558266],
       [0.06756713, 0.60934023, 0.04671566, 1.        , 0.11528469,
        0.4180021 , 0.9700888 , 0.36067815, 0.92797974, 0.13077514,
        0.38983956, 0.76836828, 0.26476734, 0.49291912, 0.63829914,
        0.84259661, 0.81266431, 0.9894883 , 0.55080719, 0.38321195],
       [0.26092211, 0.82916397, 0.82238936, 0.07595647, 0.74031538,
        0.36701451, 0.37550646, 0.73067131, 1.        , 0.34906704,
        0.34865619, 0.38745529, 0.21846439, 0.84449641, 0.47852762,
        0.06660899, 0.98370676, 0.03336072, 0.68515861, 0.10193445]])

In [111]:
covmat[[0, 3, 8]][:,[0, 3, 8]]

array([[1.        , 0.69208183, 0.00132104],
       [0.06756713, 1.        , 0.92797974],
       [0.26092211, 0.07595647, 1.        ]])

In [114]:
covmat
for el in indices_scatter[19][0]:
    idx = list(el)
    matrix = covmat[idx][:,idx]
    det = np.linalg.det(matrix)
    print(det)


-0.32794926646862294


In [17]:
dview.scatter('index_mat',indexes)

In [18]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


In [19]:
def det(N):
    dets = numpy.zeros(N,dtype=numpy.float32)
    #dets = []
    idx_mat = index_mat[0]
    i  0
    for ind in idx_mat:
        idx = list(ind)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        i+=1
        #dets.append(numpy.linalg.det(matrix))
    return dets

In [20]:
ar = dview.apply_async(det,int(round(n/len(rc.ids),0)))
ar.wait_interactive()

  58/58 tasks finished after    0 s
done


In [21]:
res = ar.get()

In [22]:
res[0]

array([1.0044738e+13, 1.0049905e+13, 1.0024433e+13, ..., 9.9787242e+12,
       1.0000269e+13, 1.0017512e+13], dtype=float32)

In [23]:
aux = 0
for el in res:
    aux += len(el)
    #aux += len(el)
aux

184730

In [24]:
round(n/59,0)

3131.0

In [25]:
import numpy as np

dets_iterative = np.zeros(int(n))
hola = itertools.combinations(arr, sub_mat_n)


In [26]:
%%time
i = 0
for ind in tqdm_notebook(hola):
    idx = list(ind)
    matrix = covmat[idx][:,idx]
    dets_iterative[i] = numpy.linalg.det(matrix)
    i+=1


CPU times: user 2.73 s, sys: 8 ms, total: 2.74 s
Wall time: 2.68 s


In [27]:
dets_iterative[0] 

10044737943501.139

In [28]:
res[0]

array([1.0044738e+13, 1.0049905e+13, 1.0024433e+13, ..., 9.9787242e+12,
       1.0000269e+13, 1.0017512e+13], dtype=float32)

In [29]:
ar.metadata[0]

{'msg_id': 'be9b831b-8ce25841d44a16ddfeb4744d',
 'submitted': datetime.datetime(2019, 6, 27, 4, 41, 58, 740593, tzinfo=tzutc()),
 'started': datetime.datetime(2019, 6, 27, 4, 41, 58, 777728, tzinfo=tzutc()),
 'completed': datetime.datetime(2019, 6, 27, 4, 41, 59, 91251, tzinfo=tzutc()),
 'received': datetime.datetime(2019, 6, 27, 4, 41, 59, 94022, tzinfo=tzutc()),
 'engine_uuid': 'd16c4110-ea35da15a55c6acfd3b52c03',
 'engine_id': 0,
 'follow': [],
 'after': [],
 'status': 'ok',
 'execute_input': None,
 'execute_result': None,
 'error': None,
 'stdout': '',
 'stderr': '',
 'outputs': [],
 'data': {}}